# 🏥 Fine-Tune Mistral/Llama-3 for Nursing SBAR Summarization

This notebook fine-tunes a Large Language Model (LLM) to convert clinical transcripts into SBAR (Situation, Background, Assessment, Recommendation) summaries.

**Hardware Requirement**: NVIDIA T4 GPU (Google Colab Free Tier is sufficient).

In [ ]:
# 1. Install Dependencies
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.26" trl peft accelerate bitsandbytes

In [ ]:
# 2. Load Base Model (Unsloth optimized)
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit", # or "unsloth/mistral-7b-v0.3-bnb-4bit"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

In [ ]:
# 3. Add LoRA Adapters
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [ ]:
# 4. Load Dataset (Update with YOUR repo name)
from datasets import load_dataset

# REPLACE WITH YOUR REPO NAME from the previous step
HF_REPO_NAME = "YOUR_USERNAME/nursing-sbar-instruct"

dataset = load_dataset(HF_REPO_NAME, split = "train")

# Format function
def formatting_prompts_func(examples):
    # Must match the ChatML format or similar
    convos = examples["messages"]
    texts = []
    for convo in convos:
        # Simple Chat Format: USER: ... ASSISTANT: ...
        user_msg = convo[0]["content"]
        assistant_msg = convo[1]["content"]
        text = f"<|user|>\n{user_msg}\n<|assistant|>\n{assistant_msg}<|endoftext|>"
        texts.append(text)
    return { "text" : texts, }

dataset = dataset.map(formatting_prompts_func, batched = True)

In [ ]:
# 5. Train
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60, # Adjust based on dataset size (e.g. 1 epoch)
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

trainer.train()

In [ ]:
# 6. Inference Test
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

inputs = tokenizer(
[
    "<|user|>\nTranscript: I have a headache and my BP is 150/90. What should I do?\n<|assistant|>"
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
print(tokenizer.batch_decode(outputs))

In [ ]:
# 7. Save Model to Hugging Face
# Authorization
from huggingface_hub import login
login("YOUR_HF_TOKEN_HERE")

model.push_to_hub_merged("YOUR_USERNAME/nursing-llama3-sbar", tokenizer, save_method = "lora")
# Or save locally:
# model.save_pretrained("lora_model")